Copy of Train_4

#Imports

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Dense, Dropout, Flatten, Reshape
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, LearningRateScheduler
from keras.regularizers import l2
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import math

#Definitions (Paths, Optimizer, DataGen)

In [ ]:
# ================ PATH DEFINITIONS ================
train_data_dir = '/content/drive/MyDrive/ITI_GP_ST/FinalModel/Datasets/AccidentDetection/dataset_preaug/train_preaug'
validation_data_dir = '/content/drive/MyDrive/ITI_GP_ST/Datasets/AccidentDetection/val'


# ================ OPTIMIZER DEFINITION ================
optimizer = Adam(learning_rate=0.01)

# ================ DATAGEN DEFINITIONS ================
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range = 15,
    horizontal_flip = True,
    zoom_range = 0.2,
    shear_range = 0.1,
    fill_mode = 'reflect',
    width_shift_range = 0.1,
    height_shift_range = 0.1
)

val_datagen = ImageDataGenerator(rescale=1./255)

# ================ TRAIN AND VAL GENERATOR FOR THE CNN ================
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(256, 256),
    color_mode='grayscale',
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(256, 256),
    color_mode='grayscale',
    batch_size=32,
    class_mode='binary'
)

Found 4512 images belonging to 2 classes.
Found 739 images belonging to 2 classes.


#Model

##Layering

In [ ]:
classifier_1 = Sequential()

classifier_1.add(Conv2D(64,(3,3),activation='relu', input_shape=(256,256,1)))
classifier_1.add(MaxPooling2D(pool_size=(2,2)))
classifier_1.add(BatchNormalization())
classifier_1.add(Dropout(0.2))

# Bloack 1
classifier_1.add(Conv2D(16,(1,1),activation='relu'))
classifier_1.add(Conv2D(64,(3,3),activation='relu'))
classifier_1.add(MaxPooling2D(pool_size=(2,2)))
classifier_1.add(BatchNormalization())
classifier_1.add(Dropout(0.2))

# Block 2
classifier_1.add(Conv2D(16,(1,1),activation='relu'))
classifier_1.add(Conv2D(64,(3,3),activation='relu'))
classifier_1.add(MaxPooling2D(pool_size=(2,2)))
classifier_1.add(BatchNormalization())
classifier_1.add(Dropout(0.2))

# Block 3
classifier_1.add(Conv2D(16,(1,1),activation='relu'))
classifier_1.add(Conv2D(64,(3,3),activation='relu'))
classifier_1.add(Conv2D(16,(1,1),activation='relu'))
classifier_1.add(Conv2D(64,(3,3),activation='relu'))
classifier_1.add(MaxPooling2D(pool_size=(2,2)))
classifier_1.add(BatchNormalization())
classifier_1.add(Dropout(0.2))

classifier_1.add(Conv2D(16,(1,1),activation='relu'))
classifier_1.add(MaxPooling2D(pool_size=(2,2)))
classifier_1.add(BatchNormalization())

# Fully Connected layers
classifier_1.add(Flatten())
classifier_1.add(Dense(128,activation='relu'))
classifier_1.add(Dropout(0.2))

# Output layer
classifier_1.add(Dense(1,activation='sigmoid'))

classifier_1.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
classifier_1.summary()

# ================ CHECKPOINT FOR AUTOENCODER ================
checkpoint_1 = ModelCheckpoint('/content/drive/MyDrive/ITI_GP_ST/ourTraining/4_newModel_conv1x1/NewDataset/SavedWeights/epoch_{epoch:02d}.weights.h5', save_weights_only=True, save_freq='epoch')


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 254, 254, 64)   │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 127, 127, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 127, 127, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 127, 127, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 127, 127, 16)   │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 125, 125, 64)   │         9,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 62, 62, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 62, 62, 16)     │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 60, 60, 64)     │         9,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 30, 30, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 30, 30, 16)     │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 28, 28, 64)     │         9,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 28, 28, 16)     │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 26, 26, 64)     │         9,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 13, 13, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 13, 13, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 13, 13, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 13, 13, 16)     │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 6, 6, 16)       │             

 Total params: 118,033 (461.07 KB)

 Trainable params: 117,489 (458.94 KB)

 Non-trainable params: 544 (2.12 KB)

##Training

In [ ]:
checkpoint_1 = ModelCheckpoint('/content/drive/MyDrive/ITI_GP_ST/ourTraining/4_newModel_conv1x1/NewDataset_Final_xtraPool/SavedWeights/epoch_{epoch:02d}.weights.h5', save_weights_only=True, save_freq='epoch')

In [ ]:
classifier_1.save('/content/drive/MyDrive/ITI_GP_ST/ourTraining/4_newModel_conv1x1/NewDataset_Final_xtraPool/SavedModels/Conv1x1_Dense128_seq_newDataset_xtraPool.h5')

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_accuracy',
                                            patience=2,
                                            factor=0.5,
                                            min_lr = 0.00001,
                                            verbose = 1)

In [ ]:
# ================= TRAINING ================
history = classifier_1.fit(
    train_generator,
    epochs=30,
    validation_data=val_generator,
    callbacks=[learning_rate_reduction, checkpoint_1]
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 2721s 19s/step - accuracy: 0.6196 - loss: 0.8563 - val_accuracy: 0.4912 - val_loss: 0.9280 - learning_rate: 0.0100
Epoch 2/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 82s 582ms/step - accuracy: 0.7252 - loss: 0.5444 - val_accuracy: 0.4912 - val_loss: 1.4222 - learning_rate: 0.0100
Epoch 3/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 84s 597ms/step - accuracy: 0.7879 - loss: 0.4685 - val_accuracy: 0.5129 - val_loss: 1.1409 - learning_rate: 0.0100
Epoch 4/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 82s 582ms/step - accuracy: 0.7866 - loss: 0.4850 - val_accuracy: 0.5413 - val_loss: 1.2199 - learning_rate: 0.0100
Epoch 5/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 81s 576ms/step - accuracy: 0.8036 - loss: 0.4289 - val_accuracy: 0.5521 - val_loss: 1.0538 - learning_rate: 0.0100
Epoch 6/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 82s 573ms/step - accuracy: 0.8196 - loss: 0.3992 - val_accuracy: 0.6008 - val_loss: 1.1118 - learning_rate: 0.0100
Epoch 7/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 83s 583ms/step - accuracy: 0.8

##Save

In [ ]:
classifier_1.save('/content/drive/MyDrive/ITI_GP_ST/ourTraining/4_newModel_conv1x1/NewDataset_Final_xtraPool/SavedModels/Conv1x1_Dense128_seq_newDataset_xtraPool_aftr.h5')

#**Testing**

##Test Generator

In [ ]:
testing_data_dir = '/content/drive/MyDrive/ITI_GP_ST/Datasets/AccidentDetection/test'

tst_datagen = ImageDataGenerator(rescale=1./255)

tst_generator = tst_datagen.flow_from_directory(
    testing_data_dir,
    target_size=(256, 256),
    color_mode='grayscale',
    batch_size=32,
    class_mode='binary'
)

Found 453 images belonging to 2 classes.


##Evaluation (test)

In [ ]:
classifier_1.load_weights('/content/drive/MyDrive/ITI_GP_ST/ourTraining/4_newModel_conv1x1/NewDataset_Final_xtraPool/SavedWeights/epoch_29.weights.h5')

In [ ]:
tst_loss, tst_accuracy = classifier_1.evaluate(tst_generator)

15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 289ms/step - accuracy: 0.7279 - loss: 0.6642


In [ ]:
print(f"Testing Accuracy: {tst_accuracy*100: .2f}%")
print(f"Testing Loss: {tst_loss*100: .2f}%")

Testing Accuracy:  71.08%
Testing Loss:  69.84%
